In [1]:
import os
import pandas as pd
import pymysql
import numpy as np
from tqdm.auto import tqdm
import warnings
import db
warnings.filterwarnings('ignore')

conn = db.getConnRSTR()
cursor = conn.cursor()


def dev_db_conn(): 
    """개발 DB 접속 함수""" 
    conn = pymysql.connect( 
        host = 'db-7ma06.pub-cdb.ntruss.com', 
        user = 'redtable', 
        password = 'fpemxpdlqmf5491!@#', 
        autocommit = True, 
        cursorclass = pymysql.cursors.DictCursor) 
    return conn

def live_db_conn():
    """Live DB 접속 함수"""
    conn = pymysql.connect(
        host = "db-6j3k3.pub-cdb.ntruss.com", 
        user = "redtable",
        password = "fpemxpdlqmf5491!@#",
        autocommit = True, 
        cursorclass = pymysql.cursors.DictCursor, 
        db = "redtable2021")
    return conn

C:\Users\레드테이블\AppData\Local\Temp\ipykernel_5504\1307375143.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## 모범음식점 전처리 진행

https://www.data.go.kr/data/15096282/standard.do
데이터 다운받아 진행
* 전처리 : 전화번호,날짜 년도만 따기 , 
* store_id 매핑 : 인허가번호,전화번호,업소명


In [13]:
df = pd.read_excel("LOCALDATA_ALL_12_03_01_E.xlsx")
df

,번호,업소명,인허가번호,도로명주소,소재지주소,영업상태구분코드,영업상태명,폐업일자,신청일자,지정일자,지정취소일자,지정취소사유,음식의유형,주된음식종류,재지정일자,최종수정일자,데이터갱신구분,데이터갱신일자,전화번호
0,1,청진옥,19660025011,NaN,서울특별시 종로구 청진동 89-0 지상1층,2,폐업,2008-09-19,20071109,20071120,20080919.0,폐업민원처리: 200830000340003507,NaN,해장국,NaN,2008-09-19 00:00:00,I,2019-04-09 18:08:48,0207351690
1,2,한일관,19670025035,NaN,서울특별시 종로구 청진동 119-1,2,폐업,2008-08-11,20071121,20071120,20080811.0,폐업민원처리: 200830000340003020,NaN,한정식,NaN,2008-08-11 00:00:00,I,2019-04-09 18:08:48,0207323735
2,3,남대문막내횟집,19670025019,서울특별시 종로구 우정국로 4-1 (관철동),서울특별시 종로구 관철동 263-0,1,영업,NaN,20140107,20150915,NaN,NaN,일식,회,20231129.0,2023-11-29 16:28:41,U,2023-12-01 02:40:00,02 7348653
3,4,원조할머니 낙지센타,19670025022,NaN,서울특별시 종로구 청진동 265-1,1,영업,NaN,20071109,20071120,20091026.0,행정처분,NaN,낙지전골,NaN,2018-08-20 14:51:13,I,2019-04-09 18:08:48,02 7341226
4,5,함흥곰보냉면,19680025024,"서울특별시 종로구 창경궁로 109, 401,405호 (인의동)","서울특별시 종로구 인의동 112-14 401,405호",1,영업,NaN,20071112,20071120,NaN,NaN,한식,냉면,20231129.0,2023-11-29 16:14:00,U,2023-12-01 02:40:00,0222732833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64686,64687,래원,20230911276,"제주특별자치도 서귀포시 대정읍 글로벌에듀로 382, B107-B109호",제주특별자치도 서귀포시 대정읍 보성리 2476-13,1,영업,NaN,20230918,20231101,NaN,NaN,중식,"짜장, 짬뽕",NaN,2023-11-08 10:21:09,I,2023-11-10 01:13:25,NaN
64687,64688,내돈니돈,20230911296,"제주특별자치도 서귀포시 신서귀로 3, 상가동 지하 1층 B1호 (법환동)",제주특별자치도 서귀포시 법환동 748-1,1,영업,NaN,20230918,20231101,NaN,NaN,한식,"숙성흑돼지, 김치찌개",NaN,2023-11-08 10:09:06,I,2023-11-10 01:13:25,NaN
64688,64689,다다미,20230911567,"제주특별자치도 서귀포시 동홍중앙로 68, 1층 (동홍동)",제주특별자치도 서귀포시 동홍동 412-6,1,영업,NaN,20230918,20231101,NaN,NaN,일식,활어회,NaN,2023-11-08 10:30:28,I,2023-11-10 01:13:25,NaN
64689,64690,전원일기동광점,20230911715,제주특별자치도 서귀포시 안덕면 신화역사로 548,제주특별자치도 서귀포시 안덕면 동광리 533-3,1,영업,NaN,20230918,20231101,NaN,NaN,한식,돔베정식,NaN,2023-11-08 10:22:23,I,2023-11-10 01:13:25,NaN


In [14]:
df['전화번호'] = df['전화번호'].replace(" ",'', regex=True)
df['전화번호'] = df['전화번호'].replace("-",'', regex=True)


In [15]:
df['전화번호'] = df['전화번호'].astype(str)

In [16]:

def preprocess_phone_number(number):
    # 숫자만 추출
    digits = ''.join(filter(str.isdigit, number))

    # 02로 시작하는 경우
    if digits.startswith('02'):
        if len(digits) == 9:
            return f'{digits[:2]}-{digits[2:5]}-{digits[5:]}'
        elif len(digits) == 10:
            return f'{digits[:2]}-{digits[2:6]}-{digits[6:]}'
        else:
            return 'spec'
    else:
        if len(digits) == 10:
            return f'{digits[:3]}-{digits[3:6]}-{digits[6:]}'
        else:
            return 'spec'

# 전화번호 전처리
df['전화번호_전처리'] = df['전화번호'].apply(preprocess_phone_number)
df['전화번호'] = df['전화번호'].replace("nan",None, regex=True)
df

,번호,업소명,인허가번호,도로명주소,소재지주소,영업상태구분코드,영업상태명,폐업일자,신청일자,지정일자,지정취소일자,지정취소사유,음식의유형,주된음식종류,재지정일자,최종수정일자,데이터갱신구분,데이터갱신일자,전화번호,전화번호_전처리
0,1,청진옥,19660025011,NaN,서울특별시 종로구 청진동 89-0 지상1층,2,폐업,2008-09-19,20071109,20071120,20080919.0,폐업민원처리: 200830000340003507,NaN,해장국,NaN,2008-09-19 00:00:00,I,2019-04-09 18:08:48,0207351690,02-0735-1690
1,2,한일관,19670025035,NaN,서울특별시 종로구 청진동 119-1,2,폐업,2008-08-11,20071121,20071120,20080811.0,폐업민원처리: 200830000340003020,NaN,한정식,NaN,2008-08-11 00:00:00,I,2019-04-09 18:08:48,0207323735,02-0732-3735
2,3,남대문막내횟집,19670025019,서울특별시 종로구 우정국로 4-1 (관철동),서울특별시 종로구 관철동 263-0,1,영업,NaN,20140107,20150915,NaN,NaN,일식,회,20231129.0,2023-11-29 16:28:41,U,2023-12-01 02:40:00,027348653,02-734-8653
3,4,원조할머니 낙지센타,19670025022,NaN,서울특별시 종로구 청진동 265-1,1,영업,NaN,20071109,20071120,20091026.0,행정처분,NaN,낙지전골,NaN,2018-08-20 14:51:13,I,2019-04-09 18:08:48,027341226,02-734-1226
4,5,함흥곰보냉면,19680025024,"서울특별시 종로구 창경궁로 109, 401,405호 (인의동)","서울특별시 종로구 인의동 112-14 401,405호",1,영업,NaN,20071112,20071120,NaN,NaN,한식,냉면,20231129.0,2023-11-29 16:14:00,U,2023-12-01 02:40:00,0222732833,02-2273-2833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64686,64687,래원,20230911276,"제주특별자치도 서귀포시 대정읍 글로벌에듀로 382, B107-B109호",제주특별자치도 서귀포시 대정읍 보성리 2476-13,1,영업,NaN,20230918,20231101,NaN,NaN,중식,"짜장, 짬뽕",NaN,2023-11-08 10:21:09,I,2023-11-10 01:13:25,None,spec
64687,64688,내돈니돈,20230911296,"제주특별자치도 서귀포시 신서귀로 3, 상가동 지하 1층 B1호 (법환동)",제주특별자치도 서귀포시 법환동 748-1,1,영업,NaN,20230918,20231101,NaN,NaN,한식,"숙성흑돼지, 김치찌개",NaN,2023-11-08 10:09:06,I,2023-11-10 01:13:25,None,spec
64688,64689,다다미,20230911567,"제주특별자치도 서귀포시 동홍중앙로 68, 1층 (동홍동)",제주특별자치도 서귀포시 동홍동 412-6,1,영업,NaN,20230918,20231101,NaN,NaN,일식,활어회,NaN,2023-11-08 10:30:28,I,2023-11-10 01:13:25,None,spec
64689,64690,전원일기동광점,20230911715,제주특별자치도 서귀포시 안덕면 신화역사로 548,제주특별자치도 서귀포시 안덕면 동광리 533-3,1,영업,NaN,20230918,20231101,NaN,NaN,한식,돔베정식,NaN,2023-11-08 10:22:23,I,2023-11-10 01:13:25,None,spec


In [17]:
df['소재지주소'] = df['소재지주소'].astype(str)
df['AREA_1'] = df['소재지주소'].apply(lambda x : x.split(' ')[0])
df['AREA_1'].value_counts()

AREA_1
서울특별시      13679
경기도        11111
경상북도        4162
경상남도        4041
인천광역시       3626
전라남도        3568
대구광역시       3364
강원특별자치도     3121
부산광역시       2946
충청남도        2585
충청북도        2460
전북특별자치도     2235
광주광역시       1919
제주특별자치도     1908
대전광역시       1905
울산광역시       1844
세종특별자치시      160
nan           57
Name: count, dtype: int64

In [18]:
df

,번호,업소명,인허가번호,도로명주소,소재지주소,영업상태구분코드,영업상태명,폐업일자,신청일자,지정일자,...,지정취소사유,음식의유형,주된음식종류,재지정일자,최종수정일자,데이터갱신구분,데이터갱신일자,전화번호,전화번호_전처리,AREA_1
0,1,청진옥,19660025011,NaN,서울특별시 종로구 청진동 89-0 지상1층,2,폐업,2008-09-19,20071109,20071120,...,폐업민원처리: 200830000340003507,NaN,해장국,NaN,2008-09-19 00:00:00,I,2019-04-09 18:08:48,0207351690,02-0735-1690,서울특별시
1,2,한일관,19670025035,NaN,서울특별시 종로구 청진동 119-1,2,폐업,2008-08-11,20071121,20071120,...,폐업민원처리: 200830000340003020,NaN,한정식,NaN,2008-08-11 00:00:00,I,2019-04-09 18:08:48,0207323735,02-0732-3735,서울특별시
2,3,남대문막내횟집,19670025019,서울특별시 종로구 우정국로 4-1 (관철동),서울특별시 종로구 관철동 263-0,1,영업,NaN,20140107,20150915,...,NaN,일식,회,20231129.0,2023-11-29 16:28:41,U,2023-12-01 02:40:00,027348653,02-734-8653,서울특별시
3,4,원조할머니 낙지센타,19670025022,NaN,서울특별시 종로구 청진동 265-1,1,영업,NaN,20071109,20071120,...,행정처분,NaN,낙지전골,NaN,2018-08-20 14:51:13,I,2019-04-09 18:08:48,027341226,02-734-1226,서울특별시
4,5,함흥곰보냉면,19680025024,"서울특별시 종로구 창경궁로 109, 401,405호 (인의동)","서울특별시 종로구 인의동 112-14 401,405호",1,영업,NaN,20071112,20071120,...,NaN,한식,냉면,20231129.0,2023-11-29 16:14:00,U,2023-12-01 02:40:00,0222732833,02-2273-2833,서울특별시
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64686,64687,래원,20230911276,"제주특별자치도 서귀포시 대정읍 글로벌에듀로 382, B107-B109호",제주특별자치도 서귀포시 대정읍 보성리 2476-13,1,영업,NaN,20230918,20231101,...,NaN,중식,"짜장, 짬뽕",NaN,2023-11-08 10:21:09,I,2023-11-10 01:13:25,None,spec,제주특별자치도
64687,64688,내돈니돈,20230911296,"제주특별자치도 서귀포시 신서귀로 3, 상가동 지하 1층 B1호 (법환동)",제주특별자치도 서귀포시 법환동 748-1,1,영업,NaN,20230918,20231101,...,NaN,한식,"숙성흑돼지, 김치찌개",NaN,2023-11-08 10:09:06,I,2023-11-10 01:13:25,None,spec,제주특별자치도
64688,64689,다다미,20230911567,"제주특별자치도 서귀포시 동홍중앙로 68, 1층 (동홍동)",제주특별자치도 서귀포시 동홍동 412-6,1,영업,NaN,20230918,20231101,...,NaN,일식,활어회,NaN,2023-11-08 10:30:28,I,2023-11-10 01:13:25,None,spec,제주특별자치도
64689,64690,전원일기동광점,20230911715,제주특별자치도 서귀포시 안덕면 신화역사로 548,제주특별자치도 서귀포시 안덕면 동광리 533-3,1,영업,NaN,20230918,20231101,...,NaN,한식,돔베정식,NaN,2023-11-08 10:22:23,I,2023-11-10 01:13:25,None,spec,제주특별자치도


In [19]:
#phone_ret = df[(df['전화번호_전처리'] =='spec' ) & (df['전화번호'] != "None")]
phone_ret = df[(df['전화번호_전처리'] =='spec' ) & df['전화번호'].notna()]
phone_ret

,번호,업소명,인허가번호,도로명주소,소재지주소,영업상태구분코드,영업상태명,폐업일자,신청일자,지정일자,...,지정취소사유,음식의유형,주된음식종류,재지정일자,최종수정일자,데이터갱신구분,데이터갱신일자,전화번호,전화번호_전처리,AREA_1
215,216,신청록,20020025799,NaN,"서울특별시 종로구 내수동 110-5 지상1,2층",2,폐업,2011-06-03,20090831,20091028,...,폐업민원처리: 201130000340002543,NaN,NaN,NaN,2011-06-03 00:00:00,I,2019-04-09 18:08:48,7301016,spec,서울특별시
218,219,도담설렁탕,20020025920,"서울특별시 종로구 종로5길 68, 지하1층 (수송동)",서울특별시 종로구 수송동 80 지하1층,2,폐업,2024-03-04,20071109,20071120,...,영업자지위승계민원처리: 200930000340001828,NaN,샤브샤브,20080923.0,2024-03-04 16:37:56,U,2024-03-06 02:40:00,7350288,spec,서울특별시
222,223,효자동평양만두,20030025143,서울특별시 종로구 자하문로 81 (신교동),서울특별시 종로구 신교동 52,2,폐업,2013-06-21,20120914,20121127,...,폐업민원처리: 201330000340002966,한식,만두,NaN,2013-06-21 00:00:00,I,2019-04-09 18:08:48,7237115,spec,서울특별시
237,238,장모님해장국,20030026064,NaN,서울특별시 종로구 구기동 88-40,2,폐업,2012-02-08,20080701,20081006,...,행정처분,NaN,NaN,NaN,2008-10-08 16:36:07,I,2019-04-09 18:08:48,00203794294,spec,서울특별시
247,248,아리랑,20040025290,서울특별시 종로구 율곡로 217 (이화동),서울특별시 종로구 이화동 192-2,2,폐업,2017-03-08,20080701,20081006,...,행정처분,NaN,NaN,20091028.0,2009-10-30 11:08:26,I,2019-04-09 18:08:48,7648600,spec,서울특별시
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64442,64443,플렌비,20060631284,제주특별자치도 서귀포시 중문관광로 323 (중문동),제주특별자치도 서귀포시 중문동 2446-1 3층,1,영업,NaN,20070529,20070701,...,지정기준 부적합,NaN,뷔페,NaN,2020-12-15 09:55:11,I,2023-03-08 02:40:00,07089003737,spec,제주특별자치도
64623,64624,헬로우보일링크랩,20190624890,"제주특별자치도 서귀포시 이어도로 1048, 1층 (서호동)",제주특별자치도 서귀포시 서호동 128-6,1,영업,NaN,20200911,20201101,...,NaN,서양식,"보일링크랩, 샐러드",20231101.0,2023-11-05 19:24:38,U,2023-11-07 02:40:00,07081473453,spec,제주특별자치도
64634,64635,그대만에나폴리,20200624209,"제주특별자치도 서귀포시 보목로64번길 138, 1층 (보목동)",제주특별자치도 서귀포시 보목동 1405-1 1층,2,폐업,2021-03-12,20200913,20201101,...,폐업민원처리: 202165200000050800,양식,"피자, 파스타",NaN,2021-03-12 13:00:34,U,2021-03-14 02:40:00,050713495733,spec,제주특별자치도
64650,64651,반양,20200624888,"제주특별자치도 서귀포시 대정읍 중산간서로 2985, 1층",제주특별자치도 서귀포시 대정읍 무릉리 965-14,1,영업,NaN,20210917,20211101,...,NaN,서양식,"쌀국수, 똠양꿍",20231101.0,2023-11-05 20:34:13,U,2023-11-07 02:40:00,07088838545,spec,제주특별자치도


In [20]:
# 길이로 체크
phone_ret['전화번호_자릿수'] = phone_ret['전화번호'].apply(lambda x : len(x))
phone_ret['전화번호_자릿수'].value_counts()

전화번호_자릿수
7     3755
12    1126
11     549
0      440
3      103
8      102
2       86
9       30
6        5
1        2
4        1
Name: count, dtype: int64

In [21]:
phone_ret[phone_ret['전화번호_자릿수'] == 7].sample(5)

,번호,업소명,인허가번호,도로명주소,소재지주소,영업상태구분코드,영업상태명,폐업일자,신청일자,지정일자,...,음식의유형,주된음식종류,재지정일자,최종수정일자,데이터갱신구분,데이터갱신일자,전화번호,전화번호_전처리,AREA_1,전화번호_자릿수
20536,20537,경복궁삼계탕,19970193071,"인천광역시 미추홀구 매소홀로 61, 1,2층 (용현동)","인천광역시 미추홀구 용현동 627-409 1,2층",1,영업,NaN,20070625,20070716,...,NaN,샤브사브,20121120.0,2016-03-08 16:08:51,I,2019-04-09 18:08:48,8857789,spec,인천광역시,7
20634,20635,상일가든,20000193379,인천광역시 미추홀구 미추홀대로597번길 42 (주안동),인천광역시 미추홀구 주안동 1446-8,2,폐업,2021-11-15,20020220,20020417,...,NaN,돼지갈비,20050704.0,2021-12-03 10:45:54,U,2021-12-07 02:40:00,8688221,spec,인천광역시,7
24589,24590,오리마을,19950219051,NaN,광주광역시 남구 주월동 1147-11 옹정길86,2,폐업,2005-02-07,20011227,20011227,...,NaN,오리마을,20020425.0,2007-07-14 10:51:45,I,2019-04-09 18:08:48,6517492,spec,광주광역시,7
45685,45686,동흥루,19760474002,충청남도 예산군 예산읍 예산로 220-1,충청남도 예산군 예산읍 예산리 515-2,1,영업,NaN,20090717,20090731,...,중식,자장면,20151110.0,2021-09-30 11:19:43,I,2023-03-08 02:40:00,3312812,spec,충청남도,7
41513,41514,오대산비로봉식당,20050397060,강원특별자치도 평창군 진부면 오대산로 90 (외 2필지),강원특별자치도 평창군 진부면 간평리 106-17 외 2필지,1,영업,NaN,20160913,20161201,...,한식,"황태구이정식, 산채정식",20231113.0,2023-11-13 10:28:06,U,2023-11-15 02:40:00,3326597,spec,강원특별자치도,7


In [ ]:
#12자리인경우엔 4-4-4 11자리인경우엔 3-4-4 
# 7자리인경우엔 AREA_1 이 갖는 지역번호를 앞에 붙히고 , 그다음 preprocess_phone_number 를 다시 진행 

# 여기서 지역번호별 딕셔너리 만들기
# 지역번호 딕셔너리
area_codes = {
    '서울특별시': '02',
    '경기도': '031',
    '경상북도': '054',
    '경상남도': '055',
    '인천광역시': '032',
    '전라남도': '061',
    '대구광역시': '053',
    '강원특별자치도': '033',
    '부산광역시': '051',
    '충청남도': '041',
    '충청북도': '043',
    '전북특별자치도': '063',
    '광주광역시': '062',
    '제주특별자치도': '064',
    '대전광역시': '042',
    '울산광역시': '052',
    '세종특별자치시': '044'
}


def preprocess_phone_number2(row):
    # 숫자만 추출
    number = row['전화번호']
    area1 = row['AREA_1']

    if len(number) == 12:
        return f'{number[:4]}-{number[4:8]}-{number[8:]}'
    elif len(number) == 11:
        return f'{number[:3]}-{number[3:7]}-{number[7:]}'
    elif len(number) == 7 and area1 in area_codes:
        result_num =  f'{area_codes[area1]}{number}'
        print(result_num)
        return preprocess_phone_number(result_num)
    elif len(number) == 8 and area1 in area_codes:
        result_num =  f'{area_codes[area1]}{number}'
        print(result_num)
        return preprocess_phone_number(result_num)
    else:
        return 'spec'

phone_ret['전화번호_전처리'] = phone_ret.apply(preprocess_phone_number2, axis=1)

In [ ]:
phone_ret['전화번호_전처리'].value_counts()

In [ ]:
df['전화번호_전처리'].value_counts()

In [25]:
df.loc[phone_ret.index, '전화번호_전처리'] = phone_ret['전화번호_전처리']

# df 전화번호 처리 완료

## store_id 매핑

In [26]:
with live_db_conn() as conn:
    cur = conn.cursor()
    sql = """
    SELECT s.id, s.store_id, s.business2, s.business7, s2.name, s2.telephone, s2.telephone_plain
    from store_contract s
    LEFT JOIN store s2 ON s2.id = s.store_id
    where business7 is not null
    and business7 REGEXP '^[0-9]+$';"""

    cur.execute(sql)
    a = pd.DataFrame(cur.fetchall())
a

,id,store_id,business2,business7,name,telephone,telephone_plain
0,4,10002,1,6,테스트 매장,070-1234-1234,07012341234
1,9,10089,(주)가미에프엔아이,20210029720,명동집,02-3789-8292,0237898292
2,10,10088,다미,10382,다미죽,02-777-5077,027775077
3,11,10047,신화게판,19840131136,고기육,None,None
4,12,10091,서프홀릭 (SURF HOLIC),20140141176,썹버거,0507-1381-8331,050713818331
...,...,...,...,...,...,...,...
1175013,1619385,1628782,None,1720132478,빨질레리,None,None
1175014,1619386,1628783,None,38219621,듀퐁셔츠,None,None
1175015,1619389,1628787,None,1108496160,머니박스,02-761-8889,027618889
1175016,1621454,1630856,None,1829364017,부산요트투어 와이홀릭,0507-1430-5622,None


In [27]:
df['인허가번호'] = df['인허가번호'].astype(str)

In [ ]:
merge1 = pd.merge(df,a[['telephone','store_id','business7']],left_on='인허가번호',right_on='business7',how='left')
merge2 = pd.merge(merge1,a[['telephone','store_id']],left_on='전화번호_전처리',right_on='telephone',how='left')
merge2['store_id_x'].fillna(merge2['store_id_y'],inplace=True)
merge2['store_id_x'].value_counts()

In [29]:
len(df)

64691

In [ ]:
merge3 = merge2.drop_duplicates(subset=['업소명','인허가번호','전화번호'])
merge3['store_id_x'].value_counts()

In [31]:
merge3[merge3['store_id_x'].isnull()]['영업상태명'].value_counts()

영업상태명
폐업    25847
영업       25
Name: count, dtype: int64

In [32]:
pd.reset_option('display.max_info_rows')
pd.reset_option('display.max_info_columns')
merge3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64682 entries, 0 to 71005
Data columns (total 26 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   번호           64682 non-null  int64  
 1   업소명          64682 non-null  object 
 2   인허가번호        64682 non-null  object 
 3   도로명주소        48955 non-null  object 
 4   소재지주소        64682 non-null  object 
 5   영업상태구분코드     64682 non-null  int64  
 6   영업상태명        64682 non-null  object 
 7   폐업일자         34172 non-null  object 
 8   신청일자         64682 non-null  int64  
 9   지정일자         64682 non-null  int64  
 10  지정취소일자       51103 non-null  float64
 11  지정취소사유       49041 non-null  object 
 12  음식의유형        21037 non-null  object 
 13  주된음식종류       61455 non-null  object 
 14  재지정일자        45164 non-null  float64
 15  최종수정일자       64682 non-null  object 
 16  데이터갱신구분      64682 non-null  object 
 17  데이터갱신일자      64682 non-null  object 
 18  전화번호         60633 non-null  object 
 19  전화번호_전처리 

In [33]:
print(len(merge3['telephone_x']))
print(len(merge3['telephone_y']))

64682
64682


In [34]:
merge3.drop(columns=['store_id_y','telephone_y'],inplace=True)

In [35]:
df1 = merge3[merge3['store_id_x'].notnull()]
df1

,번호,업소명,인허가번호,도로명주소,소재지주소,영업상태구분코드,영업상태명,폐업일자,신청일자,지정일자,...,재지정일자,최종수정일자,데이터갱신구분,데이터갱신일자,전화번호,전화번호_전처리,AREA_1,telephone_x,store_id_x,business7
2,3,남대문막내횟집,19670025019,서울특별시 종로구 우정국로 4-1 (관철동),서울특별시 종로구 관철동 263-0,1,영업,NaN,20140107,20150915,...,20231129.0,2023-11-29 16:28:41,U,2023-12-01 02:40:00,027348653,02-734-8653,서울특별시,02-734-8653,15198.0,19670025019
3,4,원조할머니 낙지센타,19670025022,NaN,서울특별시 종로구 청진동 265-1,1,영업,NaN,20071109,20071120,...,NaN,2018-08-20 14:51:13,I,2019-04-09 18:08:48,027341226,02-734-1226,서울특별시,02-734-1226,783770.0,19670025022
4,5,함흥곰보냉면,19680025024,"서울특별시 종로구 창경궁로 109, 401,405호 (인의동)","서울특별시 종로구 인의동 112-14 401,405호",1,영업,NaN,20071112,20071120,...,20231129.0,2023-11-29 16:14:00,U,2023-12-01 02:40:00,0222732833,02-2273-2833,서울특별시,02-2273-2833,12669.0,19680025024
6,7,제주도집,19690025019,"서울특별시 종로구 종로 19 (종로1가,(르메이에르타운 지상1층 110-2호))",서울특별시 종로구 종로1가 24 (르메이에르타운 지상1층 110-2호),1,영업,NaN,20080701,20081006,...,NaN,2022-04-13 09:25:39,U,2022-04-15 02:40:00,027334250,02-733-4250,서울특별시,02-733-4250,130010.0,19690025019
8,9,가봉루,19720025002,"서울특별시 종로구 세종대로23길 3, 2.3층 (세종로)",서울특별시 종로구 세종로 166-1,1,영업,NaN,20071121,20071120,...,20161205.0,2020-01-14 20:02:27,U,2020-01-16 02:40:00,0207368100,02-0736-8100,서울특별시,02-0736-8100,130183.0,19720025002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71001,64687,래원,20230911276,"제주특별자치도 서귀포시 대정읍 글로벌에듀로 382, B107-B109호",제주특별자치도 서귀포시 대정읍 보성리 2476-13,1,영업,NaN,20230918,20231101,...,NaN,2023-11-08 10:21:09,I,2023-11-10 01:13:25,None,spec,제주특별자치도,None,1062278.0,20230911276
71002,64688,내돈니돈,20230911296,"제주특별자치도 서귀포시 신서귀로 3, 상가동 지하 1층 B1호 (법환동)",제주특별자치도 서귀포시 법환동 748-1,1,영업,NaN,20230918,20231101,...,NaN,2023-11-08 10:09:06,I,2023-11-10 01:13:25,None,spec,제주특별자치도,None,1060480.0,20230911296
71003,64689,다다미,20230911567,"제주특별자치도 서귀포시 동홍중앙로 68, 1층 (동홍동)",제주특별자치도 서귀포시 동홍동 412-6,1,영업,NaN,20230918,20231101,...,NaN,2023-11-08 10:30:28,I,2023-11-10 01:13:25,None,spec,제주특별자치도,None,1119895.0,20230911567
71004,64690,전원일기동광점,20230911715,제주특별자치도 서귀포시 안덕면 신화역사로 548,제주특별자치도 서귀포시 안덕면 동광리 533-3,1,영업,NaN,20230918,20231101,...,NaN,2023-11-08 10:22:23,I,2023-11-10 01:13:25,None,spec,제주특별자치도,None,1151606.0,20230911715


## rstr id 매핑
store_id 이용

In [36]:
with dev_db_conn() as conn:
    cursor = conn.cursor()
    sql = f"""
    SELECT T3.SRC_ID AS 'store_id',  T3.RSTR_ID , T2.FOODSAFETY_ID_NEW
	FROM redtable01.RSTR_CONN T3 
    left join redtable01.RSTR T2 ON T2.id = T3.RSTR_ID
   WHERE T3.SRC = 'live_db' AND T3.SRC_ID IN {tuple(df1["store_id_x"].tolist())}
    """
    cursor.execute(sql)
    RSTR_CONN = cursor.fetchall()
RSTR_CONN = pd.DataFrame(RSTR_CONN)
RSTR_CONN

,store_id,RSTR_ID,FOODSAFETY_ID_NEW
0,10012,361459,401564.0
1,10014,630004,459406.0
2,10018,5216,493325.0
3,10024,225757,561015.0
4,10026,265679,240162.0
...,...,...,...
38053,1196739,300234266,1181963.0
38054,1198890,300236418,1185600.0
38055,1201448,300238976,1188179.0
38056,1202664,300240200,1192461.0


### 날짜 분리하기

In [53]:

df2 = pd.merge(df1,RSTR_CONN,left_on='store_id_x',right_on='store_id',how='left')
df2['지정일자_year'] = df2['지정일자'].apply(lambda x : str(x)[:4])
df2['재지정일자_year'] = df2['재지정일자'].apply(lambda x : str(x)[:4] if pd.notna(x) else None)

 #['RSTR_ID'] = df2['RSTR_ID'].astype(int)

In [54]:
df3 = df2[df2['재지정일자_year'].notnull()]
df3

,번호,업소명,인허가번호,도로명주소,소재지주소,영업상태구분코드,영업상태명,폐업일자,신청일자,지정일자,...,전화번호_전처리,AREA_1,telephone_x,store_id_x,business7,store_id,RSTR_ID,FOODSAFETY_ID_NEW,지정일자_year,재지정일자_year
0,3,남대문막내횟집,19670025019,서울특별시 종로구 우정국로 4-1 (관철동),서울특별시 종로구 관철동 263-0,1,영업,NaN,20140107,20150915,...,02-734-8653,서울특별시,02-734-8653,15198.0,19670025019,15198.0,1308.0,285789.0,2015,2023
2,5,함흥곰보냉면,19680025024,"서울특별시 종로구 창경궁로 109, 401,405호 (인의동)","서울특별시 종로구 인의동 112-14 401,405호",1,영업,NaN,20071112,20071120,...,02-2273-2833,서울특별시,02-2273-2833,12669.0,19680025024,12669.0,1445.0,836711.0,2007,2023
4,9,가봉루,19720025002,"서울특별시 종로구 세종대로23길 3, 2.3층 (세종로)",서울특별시 종로구 세종로 166-1,1,영업,NaN,20071121,20071120,...,02-0736-8100,서울특별시,02-0736-8100,130183.0,19720025002,130183.0,2178.0,212697.0,2007,2016
5,10,이바구,19730025016,서울특별시 종로구 인사동6길 5 (관훈동),서울특별시 종로구 관훈동 5번지,1,영업,NaN,20141117,20141230,...,spec,서울특별시,None,14685.0,19730025016,14685.0,2457.0,481983.0,2014,2016
6,11,백제고기나라,19740025017,"서울특별시 종로구 돈화문로11길 41 (낙원동,2층)",서울특별시 종로구 낙원동 136-0 2층,1,영업,NaN,20161004,20161205,...,02-764-7733,서울특별시,02-764-7733,784674.0,19740025017,784674.0,2869.0,441856.0,2016,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38793,64675,일품순두부오일육점,20220903633,제주특별자치도 서귀포시 516로 260 (토평동),제주특별자치도 서귀포시 토평동 2017-9,1,영업,NaN,20220920,20221101,...,064-762-8850,제주특별자치도,064-762-8850,1006966.0,20220903633,1006966.0,300065184.0,1020262.0,2022,2023
38794,64676,중문삼다통갈치,20220903677,"제주특별자치도 서귀포시 색달중앙로 23, 1층 (색달동)",제주특별자치도 서귀포시 색달동 2276-5,1,영업,NaN,20220920,20221101,...,064-738-1960,제주특별자치도,064-738-1960,1007822.0,20220903677,1007822.0,300066089.0,1021263.0,2022,2023
38798,64680,대가중화요리,20220903828,"제주특별자치도 서귀포시 안덕면 화순로 154-21, 1층",제주특별자치도 서귀포시 안덕면 화순리 1053-70,1,영업,NaN,20220920,20221101,...,spec,제주특별자치도,None,1003640.0,20220903828,1003640.0,300061811.0,1016613.0,2022,2023
38799,64681,중문흑돼지천국(본점),20220903834,제주특별자치도 서귀포시 천제연로 107 (색달동),제주특별자치도 서귀포시 색달동 2480-4,1,영업,NaN,20220920,20221101,...,064-739-9036,제주특별자치도,064-739-9036,1007823.0,20220903834,1007823.0,300066090.0,1021264.0,2022,2023


In [55]:
with dev_db_conn() as conn:
    for _, row in df3.iterrows():
        cursor = conn.cursor()

        #print(row)
        row.replace({pd.NA: None}, inplace=True)
        '''cursor.execute(f" SELECT * FROM IA_KTC_SOURCE where iapIsdAcnoEcyVl = %s and cardNo  = %s", (row['iaurora_id'],row['masking_card_no']))
        exis = cursor.fetchone()

        if exis:
            pass
        else :'''

        sql = """
        INSERT INTO redtable01.RSTR_AWARD(RSTR_ID, foodsafety_id, TYPE_NM, CNFER_INSTT_NM, YEAR, OGN_NM, OGN_URL)
        VALUES (%s,%s,%s,%s,%s,%s,%s)"""
        val = (row['RSTR_ID'],row['FOODSAFETY_ID_NEW'], "모범음식점", "행정안전부", row['재지정일자_year'],"행정안전부", "https://www.localdata.go.kr/lif/lifeMFoodMapDataView.do?menuNo=40002")
        cursor.execute(sql, val)


In [57]:
df3.head(2)

,번호,업소명,인허가번호,도로명주소,소재지주소,영업상태구분코드,영업상태명,폐업일자,신청일자,지정일자,...,전화번호_전처리,AREA_1,telephone_x,store_id_x,business7,store_id,RSTR_ID,FOODSAFETY_ID_NEW,지정일자_year,재지정일자_year
0,3,남대문막내횟집,19670025019,서울특별시 종로구 우정국로 4-1 (관철동),서울특별시 종로구 관철동 263-0,1,영업,NaN,20140107,20150915,...,02-734-8653,서울특별시,02-734-8653,15198.0,19670025019,15198.0,1308.0,285789.0,2015,2023
2,5,함흥곰보냉면,19680025024,"서울특별시 종로구 창경궁로 109, 401,405호 (인의동)","서울특별시 종로구 인의동 112-14 401,405호",1,영업,NaN,20071112,20071120,...,02-2273-2833,서울특별시,02-2273-2833,12669.0,19680025024,12669.0,1445.0,836711.0,2007,2023


In [ ]:
df2['지정일자_year'].isnull()
#df2[df2['지정일자_year'].notnull()]

In [68]:
df3[df3['store_id'] == 960656] #6467

#df3.iloc[6466:]

,번호,업소명,인허가번호,도로명주소,소재지주소,영업상태구분코드,영업상태명,폐업일자,신청일자,지정일자,...,전화번호_전처리,AREA_1,telephone_x,store_id_x,business7,store_id,RSTR_ID,FOODSAFETY_ID_NEW,지정일자_year,재지정일자_year
6467,12540,계승집,20220130451,"서울특별시 강남구 학동로2길 55, 환희빌딩 지상1층 (논현동)",서울특별시 강남구 논현동 163-12 환희빌딩,1,영업,NaN,20221104,20221104,...,02-511-7829,서울특별시,02-511-7829,960656.0,20220130451,960656.0,300013552.0,964760.0,2022,2023


In [72]:
df3 = df3[df3['store_id'].notnull()]

In [73]:
with live_db_conn() as conn:
    for _, row in df3.iterrows():
        cursor = conn.cursor()

        #print(row)
        row.replace({pd.NA: None}, inplace=True)
        cursor.execute(f" SELECT * FROM redtable2021.store_award where store_id = %s and admin_id  = %s and year  = %s and type  = %s", (row['store_id'], 182 ,row['재지정일자_year'],"모범음식점"))
        exis = cursor.fetchone()

        if exis:
            pass
        else :
            sql = """
            INSERT INTO redtable2021.store_award(store_id, admin_id , year, type, company)
            VALUES (%s,%s,%s,%s,%s)"""
            val = (row['store_id'], 182, row['재지정일자_year'], "모범음식점", "행정안전부")
            cursor.execute(sql, val)


### 원본데이터 업로드

In [74]:
df4 = pd.read_excel("LOCALDATA_ALL_12_03_01_E (1).xlsx")
df4

,ID,SVC_ID,AREA_CODE,MNG_NUM,RSTR_NM,LCNS_NO,ADDR_ROAD,ADRES,OPEN_CODE,OPEN_TYPE,...,REGIST_DT,CANCLE_DT,CANCLE_TYPE,CTGRY,MAIN_MENU,RE_REGIST_DT,UPDATE_DT,RENEW_TYPE,RENEW_DT,TEL_NO
0,1,12_03_01_E,3000000,3000000-101-1966-00561,청진옥,19660025011,NaN,서울특별시 종로구 청진동 89-0 지상1층,2,폐업,...,20071120,20080919.0,폐업민원처리: 200830000340003507,NaN,해장국,NaN,2008-09-19 00:00:00,I,2019-04-09 18:08:48,0207351690
1,2,12_03_01_E,3000000,3000000-101-1967-01493,한일관,19670025035,NaN,서울특별시 종로구 청진동 119-1,2,폐업,...,20071120,20080811.0,폐업민원처리: 200830000340003020,NaN,한정식,NaN,2008-08-11 00:00:00,I,2019-04-09 18:08:48,0207323735
2,3,12_03_01_E,3000000,3000000-101-1967-03256,남대문막내횟집,19670025019,서울특별시 종로구 우정국로 4-1 (관철동),서울특별시 종로구 관철동 263-0,1,영업,...,20150915,NaN,NaN,일식,회,20231129.0,2023-11-29 16:28:41,U,2023-12-01 02:40:00,02 7348653
3,4,12_03_01_E,3000000,3000000-101-1967-04691,원조할머니 낙지센타,19670025022,NaN,서울특별시 종로구 청진동 265-1,1,영업,...,20071120,20091026.0,행정처분,NaN,낙지전골,NaN,2018-08-20 14:51:13,I,2019-04-09 18:08:48,02 7341226
4,5,12_03_01_E,3000000,3000000-101-1968-06103,함흥곰보냉면,19680025024,"서울특별시 종로구 창경궁로 109, 401,405호 (인의동)","서울특별시 종로구 인의동 112-14 401,405호",1,영업,...,20071120,NaN,NaN,한식,냉면,20231129.0,2023-11-29 16:14:00,U,2023-12-01 02:40:00,0222732833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64686,64687,12_03_01_E,6520000,6520000-101-2023-00101,래원,20230911276,"제주특별자치도 서귀포시 대정읍 글로벌에듀로 382, B107-B109호",제주특별자치도 서귀포시 대정읍 보성리 2476-13,1,영업,...,20231101,NaN,NaN,중식,"짜장, 짬뽕",NaN,2023-11-08 10:21:09,I,2023-11-10 01:13:25,NaN
64687,64688,12_03_01_E,6520000,6520000-101-2023-00110,내돈니돈,20230911296,"제주특별자치도 서귀포시 신서귀로 3, 상가동 지하 1층 B1호 (법환동)",제주특별자치도 서귀포시 법환동 748-1,1,영업,...,20231101,NaN,NaN,한식,"숙성흑돼지, 김치찌개",NaN,2023-11-08 10:09:06,I,2023-11-10 01:13:25,NaN
64688,64689,12_03_01_E,6520000,6520000-101-2023-00217,다다미,20230911567,"제주특별자치도 서귀포시 동홍중앙로 68, 1층 (동홍동)",제주특별자치도 서귀포시 동홍동 412-6,1,영업,...,20231101,NaN,NaN,일식,활어회,NaN,2023-11-08 10:30:28,I,2023-11-10 01:13:25,NaN
64689,64690,12_03_01_E,6520000,6520000-101-2023-00281,전원일기동광점,20230911715,제주특별자치도 서귀포시 안덕면 신화역사로 548,제주특별자치도 서귀포시 안덕면 동광리 533-3,1,영업,...,20231101,NaN,NaN,한식,돔베정식,NaN,2023-11-08 10:22:23,I,2023-11-10 01:13:25,NaN


In [77]:
a = df4.columns
a = a

Index(['ID', 'SVC_ID', 'AREA_CODE', 'MNG_NUM', 'RSTR_NM', 'LCNS_NO',
       'ADDR_ROAD', 'ADRES', 'OPEN_CODE', 'OPEN_TYPE', 'CLOSE_DT', 'APPN_DT',
       'REGIST_DT', 'CANCLE_DT', 'CANCLE_TYPE', 'CTGRY', 'MAIN_MENU',
       'RE_REGIST_DT', 'UPDATE_DT', 'RENEW_TYPE', 'RENEW_DT', 'TEL_NO'],
      dtype='object')

In [86]:
ds = []
for i in a:
    d=f"row['{i}']"
    print(d,",")

row['ID'] ,
row['SVC_ID'] ,
row['AREA_CODE'] ,
row['MNG_NUM'] ,
row['RSTR_NM'] ,
row['LCNS_NO'] ,
row['ADDR_ROAD'] ,
row['ADRES'] ,
row['OPEN_CODE'] ,
row['OPEN_TYPE'] ,
row['CLOSE_DT'] ,
row['APPN_DT'] ,
row['REGIST_DT'] ,
row['CANCLE_DT'] ,
row['CANCLE_TYPE'] ,
row['CTGRY'] ,
row['MAIN_MENU'] ,
row['RE_REGIST_DT'] ,
row['UPDATE_DT'] ,
row['RENEW_TYPE'] ,
row['RENEW_DT'] ,
row['TEL_NO'] ,


In [88]:
len(a)

22

In [93]:
with dev_db_conn() as conn:
    for _, row in df4.iterrows():
        cursor = conn.cursor()

        #print(row)
        row.replace({pd.NA: None}, inplace=True)
        sql = """
            INSERT INTO redtable_raw_2022.mobum_restaurant_2024 (ID, SVC_ID, AREA_CODE, MNG_NUM, RSTR_NM, LCNS_NO,
                ADDR_ROAD, ADRES, OPEN_CODE, OPEN_TYPE, CLOSE_DT, APPN_DT,
                REGIST_DT, CANCEL_DT, CANCEL_TYPE, CTGRY, MAIN_MENU,
                RE_REGIST_DT, UPDATE_DT, RENEW_TYPE, RENEW_DT, TEL_NO)
                 VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
        val = (row['ID'],row['SVC_ID'] ,
                row['AREA_CODE'] ,
                row['MNG_NUM'] ,
                row['RSTR_NM'] ,
                row['LCNS_NO'] ,
                row['ADDR_ROAD'] ,
                row['ADRES'] ,
                row['OPEN_CODE'] ,
                row['OPEN_TYPE'] ,
                row['CLOSE_DT'] ,
                row['APPN_DT'] ,
                row['REGIST_DT'] ,
                row['CANCLE_DT'] ,
                row['CANCLE_TYPE'] ,
                row['CTGRY'] ,
                row['MAIN_MENU'] ,
                row['RE_REGIST_DT'] ,
                row['UPDATE_DT'] ,
                row['RENEW_TYPE'] ,
                row['RENEW_DT'] ,
                row['TEL_NO'])
        cursor.execute(sql, val)